<a href="https://colab.research.google.com/github/yejxjj/python/blob/main/grade_manager(db).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
  ##################

  #프로그램명: 성적 관리 프로그램

  #작성자: 소프트웨어학과/조예지

  #작성일: 2025/06/13

  #프로그램 설명: DB가 연동된 성적관리 프로그램
  # - DB 파일: grade.db (같은 폴더에 자동 생성)
  # - 메뉴:
  #   1) 입력  2) 삭제  3) 검색  4) 전체 출력  5) 80↑ 인원  0) 종료


  ###################
%%writefile grade_manager.py


import sqlite3      # 파이썬 기본 내장 DB 모듈
import sys

DB = "grade.db"     # DB 파일 이름 (여기 바꾸면 다른 파일에 저장됨)

# DB 초기화 (테이블 없으면 바로 만들어 줌)
def init_db():
    with sqlite3.connect(DB) as con:
        con.execute("""
        CREATE TABLE IF NOT EXISTS students (
            stud_id TEXT PRIMARY KEY,   -- 학번(문자열)
            name    TEXT,               -- 이름
            eng     INTEGER,            -- 영어 점수
            c_lang  INTEGER,            -- C 점수
            py      INTEGER,            -- 파이썬 점수
            total   INTEGER,            -- 총점
            avg     REAL,               -- 평균
            grade   TEXT,               -- 학점(A~F)
            rank    INTEGER             -- 등수
        )
        """)

# 평균 → 학점으로 바꿔주는 함수
def calc_grade(avg):
    if avg >= 90: return "A"
    if avg >= 80: return "B"
    if avg >= 70: return "C"
    if avg >= 60: return "D"
    return "F"

# 총점 순서로 다시 등수 매기기
def recalc_ranks():
    with sqlite3.connect(DB) as con:
        cur = con.execute(
            "SELECT stud_id, total FROM students ORDER BY total DESC"
        ).fetchall()
        for idx, (sid, _) in enumerate(cur, 1):
            con.execute("UPDATE students SET rank=? WHERE stud_id=?", (idx, sid))

# 1) 입력 → DB 저장 → 바로 표 출력
def insert_and_show():
    sid  = input("학번: ")
    name = input("이름: ")
    eng  = int(input("영어: "))
    c    = int(input("C   : "))
    py   = int(input("파이썬: "))

    total = eng + c + py
    avg   = total / 3
    grade = calc_grade(avg)

    with sqlite3.connect(DB) as con:
        # 같은 학번 또 넣으면 덮어쓰기
        con.execute("""
            INSERT OR REPLACE INTO students
            VALUES (?,?,?,?,?,?,?,?,NULL)
        """, (sid, name, eng, c, py, total, avg, grade))
    recalc_ranks()   # 등수 다시 계산
    show()           # 방금 내용 포함해서 전체 출력

# 2) 학번으로 삭제
def delete():
    sid = input("삭제할 학번: ")
    with sqlite3.connect(DB) as con:
        con.execute("DELETE FROM students WHERE stud_id=?", (sid,))
    recalc_ranks()

# 3) 학번 / 이름 검색
def search():
    key = input("학번 or 이름: ")
    with sqlite3.connect(DB) as con:
        rows = con.execute("""
            SELECT * FROM students
            WHERE stud_id=? OR name LIKE ?
        """, (key, f"%{key}%")).fetchall()
    show(rows)

# 4) 표  함수
def show(rows=None):
    with sqlite3.connect(DB) as con:
        if rows is None:
            rows = con.execute(
                "SELECT * FROM students ORDER BY rank"
            ).fetchall()

    # 헤더 + 구분선
    header = ("학번", "이름", "영어", "C", "파이썬", "총점", "평균", "학점", "등수")
    widths = (8, 8, 6, 5, 7, 6, 7, 5, 5)
     # 칼럼 폭
    fmt    = " ".join(f"{{:<{w}}}" for w in widths)

    print()
    # 빈 줄
    print(fmt.format(*header))
    print("-" * sum(widths))

    # 레코드 한 줄씩 출력
    for r in rows:
        stud_id, name, eng, c, py, total, avg, grade, rank = r
        print(fmt.format(
            stud_id, name, eng, c, py, total, f"{avg:.1f}", grade, rank
        ))
    print()

# 5) 평균 80 이상 학생 수 출력
def count_over80():
    with sqlite3.connect(DB) as con:
        n, = con.execute(
            "SELECT COUNT(*) FROM students WHERE avg >= 80"
        ).fetchone()
    print(f"\n평균 80점 이상 학생: {n}명\n")

# 메인
def main():
    init_db()     # 제일 먼저 DB부터 준비해야됨
    menu = {
        "1": ("입력→저장→바로확인", insert_and_show),
        "2": ("삭제",                delete),
        "3": ("검색",                search),
        "4": ("전체 출력",           show),
        "5": ("평균 80↑ 인원",       count_over80),
        "0": ("종료",                lambda: sys.exit())
    }
    while True:
        # 메뉴 보여주기
        print("\n".join(f"{k}. {v[0]}" for k, v in menu.items()))
        choice = input("선택> ")
        menu.get(choice, (None, lambda: print("🙅 잘못 고름")))[1]()

if __name__ == "__main__":
    main()


Overwriting grade_manager.py


In [ ]:
!python grade_manager.py


1. 입력→저장→바로확인
2. 삭제
3. 검색
4. 전체 출력
5. 평균 80↑ 인원
0. 종료
선택> 4

학번       이름       영어     C     파이썬     총점     평균      학점    등수   
---------------------------------------------------------
2023078083 조예지      67     86    90      243    81.0    B     1    
2023078090 가나다      65     77    88      230    76.7    C     2    
2023078084 홍길동      89     45    85      219    73.0    C     3    

1. 입력→저장→바로확인
2. 삭제
3. 검색
4. 전체 출력
5. 평균 80↑ 인원
0. 종료
선택> 1
학번: 2023078091
이름: 김김김
영어: 34
C   : 53
파이썬: 66

학번       이름       영어     C     파이썬     총점     평균      학점    등수   
---------------------------------------------------------
2023078083 조예지      67     86    90      243    81.0    B     1    
2023078090 가나다      65     77    88      230    76.7    C     2    
2023078084 홍길동      89     45    85      219    73.0    C     3    
2023078091 김김김      34     53    66      153    51.0    F     4    

1. 입력→저장→바로확인
2. 삭제
3. 검색
4. 전체 출력
5. 평균 80↑ 인원
0. 종료
선택> 3
학번 or 이름: 조예지

학번       이름       영어     C    